In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

## Clasificación

In [ ]:
np.random.seed(7)
datos = pd.DataFrame({
    'x1': np.random.normal(0, 3, size=50),
    'x2': np.random.normal(size=50)
}).assign(clase=lambda x: (x.x1 - x.x2 * 2).apply(lambda y: 1*(y + np.random.normal(0,2) < 0)))

plt.plot(datos[datos.clase == 0].x1, datos[datos.clase == 0].x2, 'rx')
plt.plot(datos[datos.clase == 1].x1, datos[datos.clase == 1].x2, 'bo')
plt.show()

In [ ]:
datos.clase.mean()

ecuación de la línea $y=mx + b$

In [ ]:
m = 1/2
b = 0

In [ ]:
plt.plot(datos[datos.clase == 0].x1, datos[datos.clase == 0].x2, 'rx')
plt.plot(datos[datos.clase == 1].x1, datos[datos.clase == 1].x2, 'bo')
plt.plot(datos.x1.sort_values(), datos.x1.sort_values() * m + b)
plt.xlim([datos.x1.min() - 0.1, datos.x1.max() + 0.1])
plt.ylim([datos.x2.min() - 0.1, datos.x2.max() + 0.1])

In [ ]:
datos = datos.assign(pred=lambda x: 1 * (x.x2 > m * x.x1 + b))
datos.head(10)

In [ ]:
(
    datos
    .groupby(['clase', 'pred'])['x1']
    .count()
    .reset_index()
    .pivot(index='clase', columns='pred', values='x1')
)

In [ ]:
(
    datos
    .groupby(['clase', 'pred'])['x1']
    .count()
    .reset_index()
    .assign(porc = lambda x: x.x1 / x.x1.sum())
)

In [ ]:
(
    datos
    .groupby(['clase', 'pred'])
    .count()
    .reset_index()
    .assign(porc = lambda x: x.x1 / x.x1.sum())
    .query('clase != pred').porc.sum()
)

In [ ]:
def califica_clasif(datos, m, b):
    datos = datos.assign(pred=lambda x: 1 * (x.x2 > m * x.x1 + b))
    return (
        datos
        .groupby(['clase', 'pred'])
        .count()
        .reset_index()
        .assign(porc = lambda x: x.x1 / x.x1.sum())
        .query('clase != pred').porc.sum()
    )

In [ ]:
puntos = 50
mesh = np.array(np.meshgrid(np.linspace(-0.5, 2, puntos), np.linspace(datos.x2.min(), datos.x2.max(), puntos)))

espacio_parametral = pd.DataFrame({
    'm': mesh[0].reshape([puntos * puntos,]),
    'b': mesh[1].reshape([puntos * puntos,])
}).assign(perdida = lambda D: D.apply(lambda H: califica_clasif(datos, H.m, H.b), axis=1))
sns.heatmap(espacio_parametral.pivot(index='b', columns='m', values='perdida'))

In [ ]:
optimos_c = {'m': espacio_parametral.sort_values(by='perdida').m.iloc[0],
          'b': espacio_parametral.sort_values(by='perdida').b.iloc[0]}

plt.plot(datos[datos.clase == 0].x1, datos[datos.clase == 0].x2, 'rx')
plt.plot(datos[datos.clase == 1].x1, datos[datos.clase == 1].x2, 'bo')
plt.plot(datos.x1.sort_values(), datos.x1.sort_values() * optimos_c['m'] + optimos_c['b'])
plt.xlim([datos.x1.min() - 0.1, datos.x1.max() + 0.1])
plt.ylim([datos.x2.min() - 0.1, datos.x2.max() + 0.1])
print(optimos_c)
plt.show()

### Precision-Recall

$$ precision = \frac{TP}{TP + FP} $$

$$ recall = \frac{TP}{TP + FN} $$

$$ f1score = \frac{precision \times recall}{precision + recall} $$

In [ ]:
m = - 0.2
b = 4.5

datos_aux = datos.assign(pred=lambda x: 1 * (x.x2 > m * x.x1 + b)).copy()
aux = (
    datos_aux
    .groupby(['clase', 'pred'])
    .count()
    .reset_index()
    .assign(porc = lambda x: x.x1 / x.x1.sum())
)

df = pd.DataFrame({'clase': [1,1,0,0], 'pred': [0,1,0,1]}).merge(aux, how='left').fillna(0)

TP = df[(df.clase == 1) & (df.pred == 1)].porc.values[0]
TN = df[(df.clase == 0) & (df.pred == 0)].porc.values[0]
FP = df[(df.clase == 0) & (df.pred == 1)].porc.values[0]
FN = df[(df.clase == 1) & (df.pred == 0)].porc.values[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(TP, TN, FP, FN)

In [ ]:
def f1_score(datos, m, b):
    datos_aux = datos.assign(pred=lambda x: 1 * (x.x2 > m * x.x1 + b)).copy()
    aux = (
        datos_aux
        .groupby(['clase', 'pred'])
        .count()
        .reset_index()
        .assign(porc = lambda x: x.x1 / x.x1.sum())
    )
    
    df = pd.DataFrame({'clase': [1,1,0,0], 'pred': [0,1,0,1]}).merge(aux, how='left').fillna(0)
    
    TP = df[(df.clase == 1) & (df.pred == 1)].porc.values[0]
    TN = df[(df.clase == 0) & (df.pred == 0)].porc.values[0]
    FP = df[(df.clase == 0) & (df.pred == 1)].porc.values[0]
    FN = df[(df.clase == 1) & (df.pred == 0)].porc.values[0]

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    return precision * recall / (precision + recall)

In [ ]:
puntos = 50
mesh = np.array(np.meshgrid(np.linspace(-0.5, 2, puntos), np.linspace(datos.x2.min(), datos.x2.max(), puntos)))

espacio_parametral = pd.DataFrame({
    'm': mesh[0].reshape([puntos * puntos,]),
    'b': mesh[1].reshape([puntos * puntos,])
}).assign(perdida = lambda D: D.apply(lambda H: f1_score(datos, H.m, H.b), axis=1))
sns.heatmap(espacio_parametral.pivot(index='b', columns='m', values='perdida'))

In [ ]:
optimos_c = {'m': espacio_parametral.sort_values(by='perdida', ascending=False).m.iloc[0],
          'b': espacio_parametral.sort_values(by='perdida', ascending=False).b.iloc[0],
            'perdida': espacio_parametral.sort_values(by='perdida', ascending=False).perdida.iloc[0]}

plt.plot(datos[datos.clase == 0].x1, datos[datos.clase == 0].x2, 'rx')
plt.plot(datos[datos.clase == 1].x1, datos[datos.clase == 1].x2, 'bo')
plt.plot(datos.x1.sort_values(), datos.x1.sort_values() * optimos_c['m'] + optimos_c['b'])
plt.xlim([datos.x1.min() - 0.1, datos.x1.max() + 0.1])
plt.ylim([datos.x2.min() - 0.1, datos.x2.max() + 0.1])
print(optimos_c)
plt.show()

## Regresión

In [ ]:
plt.plot(datos.x1, datos.x2, 'rx')
plt.plot(datos.x1, datos.x2, 'bo')

In [ ]:
def distancia_linea(x, m, b):
    return abs(b + m * x[0] - x[1])

def perdida_superficie(m, b):
    dist = np.zeros([x_1.shape[0], x_2.shape[0]])

    for i in range(dist.shape[0]):
        for j in range(dist.shape[1]):
            dist[i, j] = distancia_linea([x_1[i], x_2[j]], m, b)
    
    plt.imshow(np.flip(dist, axis=0))

In [ ]:
x_1 = np.linspace(datos.x1.min(), datos.x1.max(), 1000)
x_2 = np.linspace(datos.x2.min(), datos.x2.max(), 1000)

perdida_superficie(-4, 1)

In [ ]:
def sum_perdida(datos, m, b):
    return datos.apply(lambda x: distancia_linea([x.x1, x.x2], m, b), axis=1).sum()

In [ ]:
puntos = 50

mesh = np.array(np.meshgrid(np.linspace(-1.5, 1.5, puntos), np.linspace(datos.x2.min(), datos.x2.max(), puntos)))

In [ ]:
espacio_parametral = pd.DataFrame({
    'm': mesh[0].reshape([puntos * puntos,]),
    'b': mesh[1].reshape([puntos * puntos,])
}).assign(perdida = lambda D: D.apply(lambda H: sum_perdida(datos, H.m, H.b), axis=1))
sns.heatmap(espacio_parametral.pivot(index='b', columns='m', values='perdida'))

In [ ]:
espacio_parametral.sort_values(by='perdida').head()

In [ ]:
optimos_r = {'m': espacio_parametral.sort_values(by='perdida').m.iloc[0],
          'b': espacio_parametral.sort_values(by='perdida').b.iloc[0]}

plt.plot(datos.x1, datos.x2, 'rx')
plt.plot(datos.x1, datos.x2, 'bo')
plt.plot(datos.x1.sort_values(), datos.x1.sort_values() * optimos_r['m'] + optimos_r['b'])
plt.xlim([datos.x1.min() - 0.1, datos.x1.max() + 0.1])
plt.ylim([datos.x2.min() - 0.1, datos.x2.max() + 0.1])

## Curva ROC

* Eje x: 1 – especificidad (= porcentaje de falsos positivos = FP/(FP+TN))
* Eje y: sensibilidad (= porcentaje de verdaderos positivos = TP/(TP+FN))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
datos.head()

In [ ]:
lr = LogisticRegression()

lr_fitted = lr.fit(datos[['x1', 'x2']], datos.clase)

In [ ]:
datos['lr_proba'] = lr_fitted.predict_proba(datos[['x1', 'x2']])[:,1]

In [ ]:
datos.head()

In [ ]:
plt.plot(datos.sort_values(by='lr_proba', ascending=False).clase.cumsum().values)
plt.plot([0, datos.shape[0]], [0, 31])

In [ ]:
def puntos_ROC(punto_corte, variable):
    datos_aux = datos.assign(pred=lambda x: 1 * (x[variable] > punto_corte)).copy()
    aux = (
        datos_aux
        .groupby(['clase', 'pred'])
        .count()
        .reset_index()
        .assign(porc = lambda x: x.x1 / x.x1.sum())
    )

    df = pd.DataFrame({'clase': [1,1,0,0], 'pred': [0,1,0,1]}).merge(aux, how='left').fillna(0)

    TP = df[(df.clase == 1) & (df.pred == 1)].porc.values[0]
    TN = df[(df.clase == 0) & (df.pred == 0)].porc.values[0]
    FP = df[(df.clase == 0) & (df.pred == 1)].porc.values[0]
    FN = df[(df.clase == 1) & (df.pred == 0)].porc.values[0]

    x = FP/(FP+TN)
    y = TP/(TP+FN)
    return x, y

In [ ]:
X = []
Y = []
pc = np.linspace(0 , 1, 100)

for punto in pc:
    x, y = puntos_ROC(punto, 'lr_proba')
    X.append(x)
    Y.append(y)
    
roc = pd.DataFrame({
    'punto_corte': pc,
    '1-especificidad': X,
    'sensibilidad': Y
})
roc.plot(x='1-especificidad', y='sensibilidad')

In [ ]:
np.random.seed(7)
datos = pd.DataFrame({
    'x1': np.random.normal(0, 3, size=10000),
    'x2': np.random.normal(size=10000)
}).assign(clase=lambda x: (x.x1 - x.x2 * 2).apply(lambda y: 1*(y + np.random.normal(0,2) < 0)))

plt.plot(datos[datos.clase == 0].x1, datos[datos.clase == 0].x2, 'rx')
plt.plot(datos[datos.clase == 1].x1, datos[datos.clase == 1].x2, 'bo')
plt.show()

In [ ]:
lr_fitted = lr.fit(datos[['x1', 'x2']], datos.clase)
datos['lr_proba'] = lr_fitted.predict_proba(datos[['x1', 'x2']])[:,1]

In [ ]:
X = []
Y = []
pc = np.linspace(0 , 1, 100)

for punto in pc:
    x, y = puntos_ROC(punto, 'lr_proba')
    X.append(x)
    Y.append(y)
    
roc = pd.DataFrame({
    'punto_corte': pc,
    '1-especificidad': X,
    'sensibilidad': Y
})
roc.plot(x='1-especificidad', y='sensibilidad')
plt.plot([0,1], [0,1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, min_samples_split=15)
rf_fitted = rf.fit(datos[['x1', 'x2']], datos.clase)

In [ ]:
datos['rf_proba'] = rf_fitted.predict_proba(datos[['x1', 'x2']])[:,1]

In [ ]:
for modelo in ['lr_proba', 'rf_proba']:
    X = []
    Y = []
    pc = np.linspace(0 , 1, 100)

    for punto in pc:
        x, y = puntos_ROC(punto, modelo)
        X.append(x)
        Y.append(y)

    roc = pd.DataFrame({
        'punto_corte': pc,
        '1-especificidad': X,
        'sensibilidad': Y
    })
    plt.plot(roc['1-especificidad'], roc['sensibilidad'], label=modelo)
plt.plot([0,1], [0,1])
plt.legend()
plt.show()